In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [215]:
import io
import requests
import numpy as np
from PIL import Image
from scipy import ndimage, misc

In [ ]:
lly = 51.65238122272299
llx = 6.319560239611333
ury = 51.77703215567637
urx = 6.53310699254102
geometry = ee.Geometry.Rectangle([llx, lly, urx, ury], None, False)

In [218]:
def getNDVI(image):
    # Normalized difference vegetation index (NDVI)
    ndvi = image.normalizedDifference(['B8','B4']).rename("NDVI")
    image = image.addBands(ndvi)
    return(image)

def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [235]:
collection = (ee.ImageCollection("COPERNICUS/S2")
              # Select the Red, Green and Blue image bands, as well as the cloud masking layer.
              .select(['B4', 'B3', 'B2', 'B8'])
              # Filter for images within a given date range.
              .filterDate('2017-06-01', '2021-08-31')
              # Filter for images that overlap with the assigned geometry.
              .filterBounds(geometry)
              # Filter for images that have less then 20% cloud coverage.
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
              .sort('CLOUDY_PIXEL_PERCENTAGE')
              .map(getNDVI)
              .map(addDate))

In [236]:
collection_size = collection.size().getInfo()
print("Number of images:", collection_size)

Number of images: 343


In [ ]:
collection_list = collection.toList(collection.size())
date_list = []
water_data = []

for index in range(collection_size):           
    image = ee.Image(collection_list.get(index))
    # Get image date
    date_url = image.getDownloadUrl({'bands': ['date'], 'region': geometry, 'scale': 10, 'format': 'NPY'})
    date_response = requests.get(date_url)
    date = np.load(io.BytesIO(date_response.content)).astype(np.int)[0][0]
    if date in date_list:
        continue
    date_list.append(date)
    # Get RGB image respon
    url = image.getThumbUrl({"min": 0.0, "max": 2000, "bands": ['B4', 'B3', 'B2'], "region": geometry, "scale": 10, "format": 'png'})
    response = requests.get(url)
    rgb_image = Image.open(io.BytesIO(response.content)).convert("RGB")
    # Get NDVI layer
    nvid_url = image.getDownloadUrl({'bands': ['NDVI'], 'region': geometry, 'scale': 10, 'format': 'NPY'})
    nvid_response = requests.get(nvid_url)
    data = np.load(io.BytesIO(nvid_response.content)).astype(np.float)
    # Process NDVI
    threshold = 0
    data[data > threshold] = 0
    data[data < threshold] = 1
    data = ndimage.maximum_filter(data, size=3)
    roi = 400 
    data = data[roi:-roi, roi:-roi]
    water_pixels = np.count_nonzero(data)
    nvid_image = Image.fromarray((data*255).astype(np.uint8))
    
    nvid_image.convert("L").save(f"river/nvid_segmented_river_{date}_water_pixels_{water_pixels}.png")
    rgb_image.save(f"river/image_{date}_water_pixels_{water_pixels}.png")

print(date_list)
print(water_data)

In [138]:
tile = collection.first()

In [137]:
from IPython.display import Image

# Create a URL to the styled image for a region around France.
url = tile.getThumbUrl({"min": 0.0, "max": 2000, "bands": ['B4', 'B3', 'B2'], "region": geometry, "scale": 10, "format": 'png'})
print(url)

# Display the thumbnail land surface temperature in France.
print('\nPlease wait while the thumbnail loads, it may take a moment...')
Image(url=url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/a4e9e7c536b84b4c25c9ff97994337dc-1550b3a4e387976dd9601413c6fc86ad:getPixels

Please wait while the thumbnail loads, it may take a moment...
